In [5]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import BatchNormalization, LeakyReLU
from keras.optimizers import Adam
from keras.datasets import mnist

# Parâmetros para as imagens
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)

# Função para construir o gerador da GAN
def build_generator():
    noise_shape = (100,)  # Dimensão do ruído de entrada
    model = Sequential()
    model.add(Dense(256, input_shape=noise_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod(img_shape), activation='tanh'))  # Saída com ativação tanh
    model.add(Reshape(img_shape))
    model.summary()

    noise = Input(shape=noise_shape)
    img = model(noise)  # Imagem gerada
    return Model(noise, img)

# Função para construir o discriminador da GAN
def build_discriminator():
    model = Sequential()
    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))  # Saída com ativação sigmoide
    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)
    return Model(img, validity)

# Função para treinar a GAN
def train(epochs, batch_size=128, save_interval=50):
    save1 = ''
    (X_train, _), (_, _) = mnist.load_data()
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3) 
    half_batch = int(batch_size / 2)
    for epoch in range(epochs):
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        imgs = X_train[idx]
        noise = np.random.normal(0, 1, (half_batch, 100))

        gen_imgs = generator.predict(noise)
        d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
        d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1))) 
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake) 

        noise = np.random.normal(0, 1, (batch_size, 100)) 
        valid_y = np.array([1] * batch_size)  # Array de uns do tamanho do lote
        g_loss = combined.train_on_batch(noise, valid_y)
        from IPython.display import clear_output
        clear_output(wait=True)
        print(save1)
        print('\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n')
        if epoch % 100 == 0:
            save1 += ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]\n" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
        if epoch % save_interval == 0:
            save_imgs(epoch)
            generator.save(f'models/generator_{epoch}_MNIST.h5')

# Função para salvar imagens geradas
def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator.predict(noise)
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/mnist_%d.png" % epoch)
    plt.close()

# Configuração do otimizador e compilação do discriminador e gerador
optimizer = Adam(0.0002, 0.5) 

discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])

generator = build_generator()
generator.compile(loss='binary_crossentropy', optimizer=optimizer)

z = Input(shape=(100,))
img = generator(z)

discriminator.trainable = False
valid = discriminator(img)
combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer='adam')  

# Treinamento da GAN
train(epochs=50000, batch_size=32, save_interval=1000)

# Salvando o modelo gerador treinado
generator.save('generator_model.h5')


0 [D loss: 0.642343, acc.: 53.12%] [G loss: 0.680123]
100 [D loss: 0.017728, acc.: 100.00%] [G loss: 6.698777]
200 [D loss: 0.036891, acc.: 100.00%] [G loss: 5.521228]
300 [D loss: 0.021968, acc.: 100.00%] [G loss: 6.103213]
400 [D loss: 0.051938, acc.: 100.00%] [G loss: 5.387980]
500 [D loss: 0.039052, acc.: 100.00%] [G loss: 4.537320]
600 [D loss: 0.085326, acc.: 100.00%] [G loss: 4.387010]
700 [D loss: 0.155657, acc.: 93.75%] [G loss: 3.908347]
800 [D loss: 0.063585, acc.: 100.00%] [G loss: 5.091537]
900 [D loss: 0.229025, acc.: 90.62%] [G loss: 5.805878]
1000 [D loss: 0.135142, acc.: 93.75%] [G loss: 3.720648]
1100 [D loss: 0.254897, acc.: 87.50%] [G loss: 5.176873]
1200 [D loss: 0.137545, acc.: 93.75%] [G loss: 4.694273]
1300 [D loss: 0.212737, acc.: 90.62%] [G loss: 5.435849]
1400 [D loss: 0.073230, acc.: 100.00%] [G loss: 4.111646]
1500 [D loss: 0.066234, acc.: 100.00%] [G loss: 5.132934]
1600 [D loss: 0.073612, acc.: 100.00%] [G loss: 4.141818]
1700 [D loss: 0.379694, acc.: 81.